In [12]:
import spacy
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def preprocess_text(text):
    """Splits text into sentences while maintaining structure and meaning."""
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    return sentences

def determine_flashcard_count(text):
    """Determines the number of flashcards based on text length."""
    word_count = len(text.split())

    if word_count < 100:
        return 3
    elif 100 <= word_count < 300:
        return 5
    elif 300 <= word_count < 600:
        return 7
    else:
        return 10

def textrank_summary(text, top_n):
    """Ranks sentences using semantic similarity and extracts the most important ones."""
    sentences = preprocess_text(text)
    if not sentences:
        return []

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences)

    similarity_matrix = cosine_similarity(embeddings)
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    return [s[1] for s in ranked_sentences[:min(top_n, len(sentences))]]

def extract_flashcards(text):
    """Generates flashcards dynamically based on text length."""
    num_flashcards = determine_flashcard_count(text)
    key_sentences = textrank_summary(text, num_flashcards)

    flashcards = {f"Flashcard {i+1}": sentence for i, sentence in enumerate(key_sentences)}
    return flashcards

# Get user input
input_text = input("Enter your text to generate flashcards:\n")

# Generate flashcards
flashcards = extract_flashcards(input_text)

# Print flashcards
print("\n📚 Generated Flashcards:\n")
for key, value in flashcards.items():
    print(f"{key}: {value}")


Enter your text to generate flashcards:
Machine learning is a subset of artificial intelligence that focuses on enabling computers to learn patterns from data. It consists of supervised, unsupervised, and reinforcement learning techniques. Supervised learning requires labeled data, while unsupervised learning finds hidden patterns in unlabeled data. Reinforcement learning is used in robotics and game AI, where agents learn by receiving rewards or penalties. The field is rapidly evolving, with applications in healthcare, finance, and automation.

📚 Generated Flashcards:

Flashcard 1: It consists of supervised, unsupervised, and reinforcement learning techniques.
Flashcard 2: Machine learning is a subset of artificial intelligence that focuses on enabling computers to learn patterns from data.
Flashcard 3: Reinforcement learning is used in robotics and game AI, where agents learn by receiving rewards or penalties.
